# 위상 정렬 (Topological Sorting)

## 줄 세우기

- 문제 출처: [백준 2252번](https://www.acmicpc.net/problem/2252)

`-` 위상 정렬 문제이다

`-` khan의 알고리즘 또는 DFS로 해결할 수 있다

`-` 여기서는 DFS로 해결하기로 함

`-` 나중에 봤을 때 이해가 안되면 알고리즘 수업 정리한거 찾아보자

`-` 참고로 모든 원소에 대해 비교한건 아니므로 한 번도 등장하지 않은 원소도 잊지말고 추가해야 한다 (어디든 상관없어서 마지막에 추가하면 된다)

In [139]:
import sys
from collections import defaultdict, deque

sys.setrecursionlimit(10**6)


def dfs(graph, node, stack):
    if node in visited:
        return
    visited[node] = True
    for next_ in graph[node]:
        if next_ not in visited:
            dfs(graph, next_, stack)
    stack.append(node)


def solution():
    global visited
    graph = defaultdict(list)
    visited = {}
    N, M = map(int, input().split())
    S = deque([])
    for _ in range(M):
        a, b = map(int, input().split())
        graph[a].append(b)
    for node in list(graph.keys()):
        stack = []
        dfs(graph, node, stack)
        S.extendleft(stack)
    for i in range(1, N + 1):
        if i not in visited:
            S.append(i)
    print(*S)


solution()

# input
# 4 2
# 4 2
# 3 1

 4 2
 4 2
 2 4


4 2 1 3


- 스택만 사용하여 제대로 구현한 버전

```python
def dfs(node, graph, stack, visited):
    if node in visited:
        return
    visited[node] = True
    for next_ in graph[node]:
        if next_ not in visited:
            dfs(next_, graph, stack, visited)
    stack.append(node)


def solution():
    global visited
    graph = defaultdict(list)
    N, M = map(int, input().split())
    stack = []
    visited = {}
    for _ in range(M):
        a, b = map(int, input().split())
        graph[a].append(b)
    for node in range(1, N + 1):
        if node in visited:
            continue
        dfs(node, graph, stack, visited)
    stack.reverse()
    print(*stack)
```

## 음악프로그램

- 문제 출처: [백준 2623번](https://www.acmicpc.net/problem/2623)

`-` 위상 정렬을 사용해 문제를 해결할 수 있다 (DFS로 구현하자)

`-` 금방 끝날줄 알았는데 생각보다 오래 걸렸다 (너무 힘들었다)

`-` 질문검색과 챗 지피티를 통해 반례를 얻었다 (루트 노드가 없는 경우, 다른 피디 같은 입력, 사이클과 정보 없는 가수가 같이 등장 등)

`-` 보조 PD가 정한 가수 순서를 그래프에 입력한 후 이를 위상 정렬하면 된다

`-` 예를 들어 담당 가수 수가 $4$이고 순서가 $6, 2, 5, 4 $라면 $6$번 노드에 $2$를 추가하고, $2$번 노드에 $5$를 추가하고, $5$번 노드에 $4$를 추가한다

`-` 사이클이 없다면 그냥 위상 정렬을 수행하면 된다 (모든 방문하지 않은 노드에 대해 DFS를 수행하여 정렬)

`-` 그런데 사이클이 있을 수 있으므로 이를 처리하기 위해 지나온 경로를 고려하기로 했다

`-` 만약 이미 지나온 경로에 해당하는 노드를 또 방문한다면 사이클이 있는 것이다

`-` 하나의 `route` 변수를 만들고 함수 시작 부분에 입력된 노드를 `route`에 추가하고 탐색 종료 부분에 입력한 노드를 다시 제거하도록 했다 (백트래킹)

`-` `visited` 변수는 방문하지 않은 변수에 DFS를 수행하기 위해 전역으로 사용한다

`-` 따라서 이전 DFS 함수 실행에서 갱신된 것이 그대로 남아있다 (재귀적으로 실행하는 것 말고 for문에서 명시적으로 실행시킨 것)

`-` 반면 `route` 변수는 임의의 노드에서 수행한 한 번의 DFS 함수 실행에서 이미 지나온 곳을 또 지나가는지 확인하기 위해 만들었다

`-` 그래서 `visited` 변수와 달리 `route` 변수는 DFS 함수가 끝나면 빈 집합이 된다 

`-` 한편, 지나온 경로를 고려하기 위해 `route` 하나의 변수만 사용하고 가수의 수가 최대 $1000$명이므로 메모리를 많이 차지하지 않는다

`-` 위와 같이 하면 사이클이 있는지 판단하면서도 위상 정렬을 제대로 수행할 수 있다

In [166]:
import sys
from collections import defaultdict

sys.setrecursionlimit(10**6)


def dfs(node, graph, stack, visited, route):
    visited[node] = True
    route.add(node)  # 이제 지나간 경로이니 추가
    for next_node in graph[node]:
        if next_node in route:
            visited[impossible_mark] = True
            return
        if next_node in visited:
            continue
        dfs(next_node, graph, stack, visited, route)
    stack.append(node)
    route.remove(node)  # 백트래킹


def solution():
    global impossible_mark
    N, M = map(int, input().split())
    impossible_mark = -1
    graph = defaultdict(set)
    for _ in range(M):
        order = list(map(int, input().split()))
        length = order[0]
        for i in range(1, length):
            s, e = order[i], order[i + 1]
            graph[s].add(e)
    route = set()  # 이미 경로에 포함된 노드를 추가하려 시도한다면 사이클이 존재하는 것이다
    visited = {}
    total_order = []
    is_impossible = False
    for node in range(1, N + 1):
        if node in visited:
            continue
        stack = []
        dfs(node, graph, stack, visited, route)
        total_order.extend(stack)
        if impossible_mark in visited:
            is_impossible = True
    total_order.reverse()
    if is_impossible:
        print(0)
        return
    for i in total_order:
        print(i)


solution()

# input
# 6 3
# 3 1 4 3
# 4 6 2 5 4
# 2 2 3

 6 3
 3 1 4 3
 4 6 2 5 4
 2 2 3


6
2
5
1
4
3


## 문제집

- 문제 출처: [백준 1766번](https://www.acmicpc.net/problem/1766)

`-` 위상 정렬 문제이다

`-` 사이클이 있는 입력은 주어지지 않는다

`-` 먼저 푸는 것이 좋은 문제를 그래프 상에 나타내야 한다 (단반향 그래프)

`-` 들어오는 간선의 개수가 $0$인 노드가 앞에 위치하되 이들 사이에도 순서가 있다

`-` 쉬운 문제를 먼저 풀어야 하므로 들어오는 간선의 개수가 $0$인 노드 중 수가 작은 것이 우선순위가 더 높다

`-` 처음에 DFS로 풀었다가 실패했다

`-` khan의 알고리즘을 사용해 문제를 해결하자

`-` khan의 알고리즘은 `queue`에서 노드를 `popleft`하여 해당 노드의 인접 노드를 탐색한다 (선입선출, 먼저 들어간 원소는 다른 원소보다 먼저 방문했다)

`-` 해당 인접 노드를 가리키는 노드는 확정되어 순서 배열에 추가한다

`-` 이는 가리켜진 노드의 `indegree`가 $1$ 감소했음을 의미하고 만약 $0$이 됐다면 `queue`에 `append`한다

`-` `queue`에 아무 원소도 남지 않을 때까지 반복하면 된다

`-` 처음에 `queue` 초기화는 `indegree`가 $0$인 노드를 `append`하면 된다 (이들 사이에 우선순위는 없으므로 추가하는 순서는 상관없다)

`-` 그렇지 않는 노드는 함수 내에서 자연스럽게 탐색하게 된다

`-` khan의 알고리즘을 통한 위상 정렬의 시간 복잡도는 $O(V+E)$이다 ($V$는 정점, $E$는 간선)

`-` 그런데 이 문제는 khan의 알고리즘을 그대로 적용하면 안된다

`-` 들어오는 간선의 개수가 $0$인 노드 사이에도 우선 순위가 존재한다

`-` 따라서 `queue` 대신에 `heap`을 사용해서 탐색 순서를 조절하자

`-` `heap`을 사용하면 들어오는 간선의 개수가 $0$인 노드 중 가장 쉬운 문제를 우선적으로 `pop` 할 수 있다

In [57]:
import heapq


def topological_sort(graph, queue):
    order = []
    while queue:
        node = heapq.heappop(queue)
        order.append(node)
        for next_node in graph[node]:
            indegrees[next_node] -= 1
            if indegrees[next_node] == 0:
                heapq.heappush(queue, next_node)
    return order


def solution():
    global indegrees
    N, M = map(int, input().split())
    graph = [set() for _ in range(N + 1)]
    indegrees = [0 for _ in range(N + 1)]
    for _ in range(M):
        s, e = map(int, input().split())
        if e not in graph[s]:
            graph[s].add(e)
            indegrees[e] += 1
    queue = []
    for v in range(1, N + 1):
        if indegrees[v] == 0:
            queue.append(v)
    heapq.heapify(queue)
    order = topological_sort(graph, queue)
    print(*order)


solution()

# input
# 6 6
# 4 1
# 4 2
# 4 3
# 3 2
# 5 6
# 4 6

 6 6
 4 1
 4 2
 4 3
 3 2
 5 6
 4 6


4 1 3 2 5 6
